# Causal impact failure modes (and how to detect them)

This notebook is designed as a **trustworthiness guide** for time-series causal impact evaluation.

We demonstrate common failure modes:
1) **Weak covariates** → poor pre-fit, unreliable counterfactual.
2) **Confounding covariates** (covariates shift at intervention) → spurious effects.
3) **Short pre-period** → unstable estimates and wide uncertainty.

We use synthetic data so we can compare the **true effect** vs the **estimated effect**.


In [ ]:
import sys
from pathlib import Path

# Ensure `src/` is importable when running from repo root
repo_root = Path.cwd()
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tecore.causal import DataSpec, ImpactConfig, ImpactMethod, run_impact
from tecore.causal.simulate_ts import SyntheticTSConfig, generate_synthetic_time_series


## Helper utilities

In [ ]:
def plot_observed_vs_counterfactual(effect_df, intervention_date, title):
    d = effect_df.copy()
    d["date"] = pd.to_datetime(d["date"])
    plt.figure()
    plt.plot(d["date"], d["y"], label="observed")
    plt.plot(d["date"], d["y_hat"], label="counterfactual")
    plt.axvline(pd.Timestamp(intervention_date), linestyle="--")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_point_effect(effect_df, intervention_date, title):
    d = effect_df.copy()
    d["date"] = pd.to_datetime(d["date"])
    plt.figure()
    plt.plot(d["date"], d["point_effect"], label="point_effect")
    plt.fill_between(d["date"], d["point_ci_low"], d["point_ci_high"], alpha=0.2, label="CI")
    plt.axhline(0.0, linestyle="--")
    plt.axvline(pd.Timestamp(intervention_date), linestyle="--")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_cum_effect(effect_df, intervention_date, title):
    d = effect_df.copy()
    d["date"] = pd.to_datetime(d["date"])
    plt.figure()
    plt.plot(d["date"], d["cum_effect"], label="cum_effect")
    plt.fill_between(d["date"], d["cum_ci_low"], d["cum_ci_high"], alpha=0.2, label="CI")
    plt.axhline(0.0, linestyle="--")
    plt.axvline(pd.Timestamp(intervention_date), linestyle="--")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

def run_ci(df, intervention_date, x_cols, bootstrap=200, block_size=7, ridge_alpha=1.0, run_placebo=True, n_placebos=25):
    spec = DataSpec(
        date_col="date",
        y_col="y",
        x_cols=x_cols,
        freq="D",
        missing_policy="raise",
        aggregation="mean",
        add_time_trend=True,
        add_day_of_week=True,
    )
    cfg = ImpactConfig(
        intervention_date=intervention_date,
        method=ImpactMethod.CAUSAL_IMPACT_LIKE,
        ridge_alpha=ridge_alpha,
        bootstrap_iters=bootstrap,
        block_size=block_size,
        alpha=0.05,
        run_placebo=run_placebo,
        n_placebos=n_placebos,
        pre_period_min_points=42,
    )
    return run_impact(df, spec, cfg)


## Scenario A: Healthy setup (strong controls, no confounding)

We generate a dataset with a real positive intervention effect and covariates that explain `y` well in the pre-period.


In [ ]:
cfg_good = SyntheticTSConfig(
    n_days=220,
    start_date="2025-01-01",
    intervention_day=140,
    level_shift=12.0,
    slope_change=0.0,
    temp_effect_amp=6.0,
    temp_effect_decay=0.10,
    confounding=False,
    random_state=10,
)
df_good, meta_good = generate_synthetic_time_series(cfg_good)
meta_good

In [ ]:
res_good = run_ci(
    df_good,
    intervention_date=meta_good["intervention_date"],
    x_cols=["sessions", "active_users", "marketing_spend", "external_index"],
    bootstrap=250,
    run_placebo=True,
    n_placebos=30,
)
res_good.summary()

In [ ]:
print("True cumulative effect (synthetic):", meta_good.get("true_cum_effect"))
print("Estimated cumulative effect:", res_good.cum_effect)
print("CI:", res_good.cum_ci)
print("Placebo p-value:", res_good.p_value)

In [ ]:
plot_observed_vs_counterfactual(res_good.effect_series, meta_good["intervention_date"], "Healthy setup: observed vs counterfactual")
plot_point_effect(res_good.effect_series, meta_good["intervention_date"], "Healthy setup: point effect")
plot_cum_effect(res_good.effect_series, meta_good["intervention_date"], "Healthy setup: cumulative effect")

## Failure mode 1: Weak covariates

We keep the *same underlying data*, but we intentionally **remove key covariates** from the model.

Expected symptoms:
- weaker pre-fit (lower R², higher RMSE)
- less stable / less credible effect estimate
- warnings triggered by diagnostics


In [ ]:
res_weak_cov = run_ci(
    df_good,
    intervention_date=meta_good["intervention_date"],
    x_cols=[],  # intentionally drop covariates, only time trend + DOW remain
    bootstrap=250,
    run_placebo=True,
    n_placebos=30,
)
res_weak_cov.summary()

In [ ]:
print("Diagnostics (good):", res_good.diagnostics)
print("Diagnostics (weak covariates):", res_weak_cov.diagnostics)
print("Warnings (weak covariates):", res_weak_cov.warnings)

In [ ]:
plot_observed_vs_counterfactual(res_weak_cov.effect_series, meta_good["intervention_date"], "Weak covariates: observed vs counterfactual")
plot_point_effect(res_weak_cov.effect_series, meta_good["intervention_date"], "Weak covariates: point effect")
plot_cum_effect(res_weak_cov.effect_series, meta_good["intervention_date"], "Weak covariates: cumulative effect")

## Failure mode 2: Confounding covariate shift

We generate a dataset with **no true treatment effect**, but with a covariate that shifts at the intervention date.

This is a classic confounding scenario: the model can produce an apparent 'effect' even when the true causal effect is zero.


In [ ]:
cfg_confound = SyntheticTSConfig(
    n_days=220,
    start_date="2025-01-01",
    intervention_day=140,
    level_shift=0.0,         # true effect = 0
    slope_change=0.0,
    temp_effect_amp=0.0,
    confounding=True,        # covariate changes at T0
    confound_shift=120.0,
    random_state=11,
)
df_confound, meta_confound = generate_synthetic_time_series(cfg_confound)
meta_confound

In [ ]:
res_confound = run_ci(
    df_confound,
    intervention_date=meta_confound["intervention_date"],
    x_cols=["sessions", "active_users", "marketing_spend", "external_index"],
    bootstrap=250,
    run_placebo=True,
    n_placebos=30,
)
res_confound.summary()

In [ ]:
print("True cumulative effect (synthetic):", meta_confound.get("true_cum_effect"))
print("Estimated cumulative effect:", res_confound.cum_effect)
print("CI:", res_confound.cum_ci)
print("Placebo p-value:", res_confound.p_value)
print("Warnings:", res_confound.warnings)

In [ ]:
plot_observed_vs_counterfactual(res_confound.effect_series, meta_confound["intervention_date"], "Confounding: observed vs counterfactual")
plot_point_effect(res_confound.effect_series, meta_confound["intervention_date"], "Confounding: point effect")
plot_cum_effect(res_confound.effect_series, meta_confound["intervention_date"], "Confounding: cumulative effect")

### Inspect covariate break at intervention

A practical guardrail: plot key covariates and check for structural breaks at the intervention date.


In [ ]:
d = df_confound.copy()
d["date"] = pd.to_datetime(d["date"])
for c in ["sessions", "active_users", "marketing_spend", "external_index"]:
    if c not in d.columns:
        continue
    plt.figure()
    plt.plot(d["date"], d[c])
    plt.axvline(pd.Timestamp(meta_confound["intervention_date"]), linestyle="--")
    plt.title(f"Confounding example: covariate '{c}'")
    plt.tight_layout()
    plt.show()

## Failure mode 3: Short pre-period

We take the healthy dataset but truncate history so the **pre-period is short**.

Expected symptoms:
- unstable fit
- wider uncertainty
- less reliable placebo inference


In [ ]:
# Truncate to keep only ~35 pre days before intervention + post days
intervention_ts = pd.Timestamp(meta_good["intervention_date"])
df_short_pre = df_good.copy()
df_short_pre["date"] = pd.to_datetime(df_short_pre["date"])
start = intervention_ts - pd.Timedelta(days=35)
df_short_pre = df_short_pre[(df_short_pre["date"] >= start)].copy()
df_short_pre["date"] = df_short_pre["date"].dt.strftime("%Y-%m-%d")

df_short_pre.head(), df_short_pre.shape

In [ ]:
# Use a smaller pre_period_min_points to allow the run, so we can see the instability
spec_short = DataSpec(
    date_col="date",
    y_col="y",
    x_cols=["sessions", "active_users", "marketing_spend", "external_index"],
    freq="D",
    add_time_trend=True,
    add_day_of_week=True,
)

cfg_short = ImpactConfig(
    intervention_date=meta_good["intervention_date"],
    method=ImpactMethod.CAUSAL_IMPACT_LIKE,
    ridge_alpha=1.0,
    bootstrap_iters=250,
    block_size=7,
    alpha=0.05,
    run_placebo=True,
    n_placebos=25,
    pre_period_min_points=20,
)

res_short_pre = run_impact(df_short_pre, spec_short, cfg_short)
res_short_pre.summary()

In [ ]:
print("Diagnostics (healthy):", res_good.diagnostics)
print("Diagnostics (short pre):", res_short_pre.diagnostics)
print("Warnings (short pre):", res_short_pre.warnings)

In [ ]:
plot_observed_vs_counterfactual(res_short_pre.effect_series, meta_good["intervention_date"], "Short pre: observed vs counterfactual")
plot_point_effect(res_short_pre.effect_series, meta_good["intervention_date"], "Short pre: point effect")
plot_cum_effect(res_short_pre.effect_series, meta_good["intervention_date"], "Short pre: cumulative effect")

## Summary: how to detect when NOT to trust the estimate

Use the built-in guardrails:

- **Pre-fit quality** (R², RMSE). Weak fit → counterfactual is unreliable.
- **Residual autocorrelation** warnings.
- **Placebo-in-time**: if your effect is common under fake dates, do not trust it.
- **Covariate break checks**: if key controls shift at intervention, confounding is likely.
- **Sensitivity tests**: if results change drastically when you drop one covariate or shorten pre, treat the estimate as fragile.


## (Optional) Save figures for an article

The cells above show charts inline. If you want reproducible files, re-run any plotting cell after setting `save_path`.
